In [55]:
import random
import numpy as np
import os
from os import path
import time
import copy
import torch
from torch import nn
from gan_training.utils4nb import return_data_test
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from gan_training import utils
from gan_training.train import Trainer, update_average
from gan_training.logger import Logger
from gan_training.checkpoints import CheckpointIO
from gan_training.inputs import get_dataset
from gan_training.distributions import get_ydist, get_zdist
from gan_training.eval_test import DisentEvaluator, Evaluator
from gan_training.config4nb import (
    load_config, build_models, build_optimizers, build_lr_scheduler,
)


In [78]:

# Arguments

#args.config_dir='./configs'
config='cells_650.yaml'
VAEname= 'FactorVAE3'
#VAEname='BetaVAE_H'
config_dir='./configs'
output_dir= './outputs/'
Model_dir='./Models'
dset_dir='./ProcessedData/'
TrainDataset='LC' 
TestDataset='LC'
Traverse=False

                   


In [79]:
#LC-141
#CellCycle-
#CPA-178-
#EMT-161-
#LiveCell - 157

VAE_name='VAE' 
GAN_name='GAN'

config_path = os.path.join(config_dir, config)
config = load_config(config_path)

nf=-1 #Base number of filters

bs=1 #'Batch size'
reg_param=-1 #'R1 regularization parameter'
w_info =-1#'weighting constant on ID Loss')
mi=max_iter=-1 #'Max training iteration')
num_workers=0 #'dataloader num_workers'
#batch_size=32 # help='batch size'
image_size= 256         # Generator Image size
datatype='rgb'
nc= 3
Traverse_SaveIm=True
c_dim= 10 #Latent synthesis size'
z_dim=256 #type=int, help='Image synthesis size')

no_cuda=False #help='Do not use cuda')
seed=1 #type=int, help='Random Seed')
infodistil_mode=True #'Infodistill')


is_cuda = (torch.cuda.is_available() and not no_cuda)


In [80]:
VAE_chkptname='last'
GAN_chkptname='model.pt'
data_dir=os.path.join(dset_dir,TestDataset)
#out_dir = os.path.join(output_dir, name)
fvae_ckpt_path = os.path.join(Model_dir, TrainDataset, VAE_name,'chkpts',VAE_chkptname)
gan_ckpt_path = os.path.join(Model_dir, TrainDataset, GAN_name,'chkpts', GAN_chkptname)
     


In [81]:
device = torch.device("cuda:0" if is_cuda else "cpu")
device

device(type='cuda', index=0)

In [82]:

# Logger
checkpoint_io = CheckpointIO(
    checkpoint_dir=fvae_ckpt_path
)

device = torch.device("cuda:0" if is_cuda else "cpu")

train_dataset = get_dataset(
    name='image',
    data_dir=data_dir,
    size=image_size,
)
train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=bs,
        num_workers=3,
        shuffle=True, pin_memory=True, sampler=None, drop_last=True
)



In [83]:

#Create models
fvae, generator, discriminator = build_models(image_size, c_dim, nc, z_dim)
data_loader = return_data_test(TestDataset, datatype, dset_dir, bs, image_size)



In [84]:
selectVAE='fvae'

if selectVAE=='dvae':
    dvae_ckpt_path =  os.path.join(Model_dir, dataset, VAE_name,'chkpts', chkptname)

    dvae_ckpt = torch.load(dvae_ckpt_path, weights_only=True)
    #for key in list(dvae_ckpt_path.keys()):
    #    dvae_ckpt_path[key.replace('decoder.', 'module.decoder.'). replace('encoder.', 'module.encoder.')] = dvae_ckpt_path.pop(key)

    dvae.load_state_dict(dvae_ckpt_path)
else:


    fvae_ckpt = torch.load(fvae_ckpt_path, weights_only=True)

    for key in list(fvae_ckpt['model_states']['VAE'].keys()):
        fvae_ckpt['model_states']['VAE'][key.replace('decode.', 'decoder.layer.').
                                         replace('decode.', 'decoder.layer.').
                                         replace('decode.', 'decoder.layer.').
                                         replace('decode.', 'decoder.layer.').
                                         replace('decode.', 'decoder.layer.').
                                         replace('encode.', 'encoder.layer.').
                                         replace('encode.', 'encoder.layer.').
                                         replace('encode.', 'encoder.layer.').
                                         replace('encode.', 'encoder.layer.').
                                         replace('encode.', 'encoder.layer.')] = fvae_ckpt['model_states']['VAE'].pop(key)


    fvae.load_state_dict(fvae_ckpt['model_states']['VAE'])

    dvae=fvae


In [85]:
# Put models on gpu if needed
dvae = dvae.to(device)
generator = generator.to(device)
discriminator = discriminator.to(device)

#g_optimizer, d_optimizer = build_optimizers(
#    generator, discriminator, dvae, config
#)

# Use multiple GPUs if possible
dvae = nn.DataParallel(dvae)
generator = nn.DataParallel(generator)
discriminator = nn.DataParallel(discriminator)

In [86]:

# Register modules to checkpoint
checkpoint_io.register_modules(
    generator=generator,
    discriminator=discriminator,
)
'''
# Logger
logger = Logger(
    log_dir=path.join(out_dir, 'logs'),
    img_dir=path.join(out_dir, 'imgs'),
    #monitoring=config['training']['monitoring'],
    #monitoring_dir=path.join(out_dir, 'monitoring')
)
'''

# Distributions
cdist = get_zdist('gauss',c_dim, device=device)
zdist = get_zdist('gauss', z_dim,
                  device=device)

 
# Save for tests
ntest = bs
#x_real, ytest = utils.get_nsamples(train_loader, ntest)
ztest = zdist.sample((ntest,))
ctest = cdist.sample((ntest,))
ztest_ = torch.cat([ztest, ctest], 1)

In [87]:

generator_test = generator

# Evaluator
dis_evaluator = DisentEvaluator(generator=generator_test, zdist=zdist, cdist=cdist,
                                batch_size=bs, device=device, dvae=dvae)
#data_loader = return_data_test()

# Train
tstart = t0 = time.time()
it = epoch_idx = -1

In [88]:
# Load checkpoint if existant
it=checkpoint_io.load(os.getcwd()+gan_ckpt_path)

# Training loop


=> Loading checkpoint...


In [89]:

mu_real_Dump=[]
mu_gen_Dump=[]
mu_real_All=[]
mu_gen_All=[]
label1=[]
label2=[]
travN_MainDir='./outputs/TraversalLoop/'
device

device(type='cuda', index=0)

In [91]:
t = time.time()

for x_true, path, label in data_loader:
    epoch_idx += 1

    #for x_real, _ in train_loader:
    
    x_true=x_true.to(device)

    mu_real, logvar, c_real = fvae(x_true, encode_only=True)
    print(c_real)
    
    recon_img=dis_evaluator.predict_image(c_real)
    recon_img=torch.stack(recon_img)
    mu_gen, logvar_gen, c_gen=fvae(recon_img, encode_only=True)
    
    mu_real_All.append(mu_real.detach().cpu())
    mu_gen_All.append(mu_gen.detach().cpu())
    
    
    mu, c_lat=dis_evaluator.predict_latent(x_true)
    if Traverse_SaveIm==True:
        if epoch_idx < 50:
            
            travN =  epoch_idx
            travN_dir = os.path.join(travN_MainDir, str(travN))
            #print('checkpoint_dir',checkpoint_dir)
            # Create missing directories
            if not os.path.exists(travN_dir):
                os.makedirs(travN_dir)
                
            c_lat_sample=c_lat
            z_sample=ztest[0]
            #c_lat_sample=c_lat_sample.unsqueeze(0)
            z_sample=z_sample.unsqueeze(0)
            travN
            ncol=dis_evaluator.traverse_c1z1(travN, z_sample, c_lat_sample, save='True', save_type='idgan', TravImRet='False', cmap='viridis')

## Sorting the feature tables according to order generated in the ImageGenerator 
    label1.append(label)
    label2.append(path)
    # do stuff
elapsed = time.time() - t
print('Time taken to complete testing this dataset=')
print(elapsed)

tensor([[-0.2313, -4.8941, -5.4866, -6.1378, -3.8961, -5.5478, -5.6597, -5.6835,
         -5.3852, -4.7717]], device='cuda:0', grad_fn=<SliceBackward0>)
tensor([[-0.3660, -5.8071, -6.3768, -7.0425, -4.8179, -5.8352, -6.1194, -6.3988,
         -5.8660, -5.4132]], device='cuda:0', grad_fn=<SliceBackward0>)
tensor([[-0.3513, -5.5143, -6.1564, -6.7223, -4.6170, -5.6533, -5.9983, -5.9248,
         -5.7349, -5.2137]], device='cuda:0', grad_fn=<SliceBackward0>)
tensor([[-0.2709, -5.0863, -5.6741, -6.2648, -4.1926, -5.5677, -5.7118, -5.6602,
         -5.5285, -4.9598]], device='cuda:0', grad_fn=<SliceBackward0>)
tensor([[-0.3222, -5.6674, -6.2680, -6.8486, -4.7067, -5.7892, -6.0771, -6.1488,
         -5.8861, -5.3444]], device='cuda:0', grad_fn=<SliceBackward0>)
tensor([[-0.3029, -5.3319, -5.9477, -6.5200, -4.3801, -5.6998, -5.8690, -5.9986,
         -5.6966, -5.0581]], device='cuda:0', grad_fn=<SliceBackward0>)
tensor([[-0.2863, -5.1653, -5.6809, -6.3718, -4.2050, -5.6214, -5.6893, -5.8950,
 

tensor([[-0.2894, -4.8738, -5.4019, -6.2317, -3.9020, -5.4960, -5.5877, -5.7851,
         -5.3323, -4.7867]], device='cuda:0', grad_fn=<SliceBackward0>)
tensor([[-0.3481, -5.7884, -6.3312, -6.9667, -4.7848, -5.8430, -6.0727, -6.3595,
         -5.8853, -5.3675]], device='cuda:0', grad_fn=<SliceBackward0>)
tensor([[-0.3170, -5.3460, -5.9296, -6.5106, -4.4557, -5.6418, -5.8535, -5.9042,
         -5.6675, -5.1334]], device='cuda:0', grad_fn=<SliceBackward0>)
tensor([[-0.2064, -4.7545, -5.6393, -6.2233, -3.5937, -5.5051, -5.9244, -5.6404,
         -5.2759, -4.7523]], device='cuda:0', grad_fn=<SliceBackward0>)
tensor([[-0.2555, -5.1631, -5.6492, -6.2853, -4.2123, -5.6718, -5.6207, -5.8739,
         -5.7331, -4.9759]], device='cuda:0', grad_fn=<SliceBackward0>)
tensor([[-0.3995, -6.0059, -6.5754, -7.2481, -4.9477, -5.9222, -6.2671, -6.5729,
         -5.9217, -5.4707]], device='cuda:0', grad_fn=<SliceBackward0>)
tensor([[-0.4616, -6.0975, -6.5600, -7.2436, -4.9962, -5.9830, -6.1854, -6.7003,
 

tensor([[-0.2272, -4.7297, -5.5085, -6.2928, -3.3713, -5.3908, -5.7433, -5.7037,
         -4.9197, -4.4362]], device='cuda:0', grad_fn=<SliceBackward0>)
tensor([[-0.3297, -5.2553, -5.5990, -6.2501, -4.3639, -5.7848, -5.7050, -6.1032,
         -5.8305, -5.0860]], device='cuda:0', grad_fn=<SliceBackward0>)
tensor([[-0.1776, -4.7565, -5.5356, -6.4700, -3.4302, -5.5112, -5.7631, -5.7433,
         -5.0858, -4.6095]], device='cuda:0', grad_fn=<SliceBackward0>)
tensor([[-0.2226, -4.7384, -5.4888, -6.3377, -3.3941, -5.4624, -5.7518, -5.7634,
         -4.9785, -4.5022]], device='cuda:0', grad_fn=<SliceBackward0>)
tensor([[-0.2428, -4.9307, -5.5103, -6.1711, -3.9083, -5.5885, -5.7021, -5.7697,
         -5.3737, -4.7166]], device='cuda:0', grad_fn=<SliceBackward0>)
tensor([[-0.2651, -5.0096, -5.5514, -6.2308, -4.0004, -5.5941, -5.6803, -5.8735,
         -5.4387, -4.7880]], device='cuda:0', grad_fn=<SliceBackward0>)
tensor([[-0.3377, -5.5599, -6.1515, -6.7909, -4.5493, -5.6280, -5.9508, -6.1767,
 

tensor([[-0.2103, -4.7409, -5.5378, -6.4374, -3.4145, -5.4186, -5.7386, -5.7427,
         -4.8679, -4.4573]], device='cuda:0', grad_fn=<SliceBackward0>)
tensor([[-0.2973, -5.0818, -5.5576, -6.2740, -4.1294, -5.6058, -5.6396, -5.8489,
         -5.4884, -4.8740]], device='cuda:0', grad_fn=<SliceBackward0>)
tensor([[-0.3220, -5.3997, -5.9588, -6.6659, -4.4057, -5.7381, -5.8892, -6.0293,
         -5.6620, -5.0952]], device='cuda:0', grad_fn=<SliceBackward0>)
tensor([[-0.5387, -6.5495, -7.2068, -7.7908, -5.4432, -6.1541, -6.7344, -7.0153,
         -6.2431, -5.8303]], device='cuda:0', grad_fn=<SliceBackward0>)
tensor([[-0.2500, -4.9053, -5.4710, -6.1680, -3.8504, -5.6139, -5.6771, -5.8210,
         -5.4405, -4.6991]], device='cuda:0', grad_fn=<SliceBackward0>)
tensor([[-0.2404, -4.9599, -5.6050, -6.2314, -3.9604, -5.5587, -5.7338, -5.6829,
         -5.4269, -4.8033]], device='cuda:0', grad_fn=<SliceBackward0>)
tensor([[-0.3768, -5.8171, -6.4532, -7.0541, -4.8562, -5.7715, -6.1503, -6.3056,
 

tensor([[-0.2793, -5.1633, -5.7742, -6.3551, -4.1862, -5.6292, -5.8538, -5.8854,
         -5.4954, -4.9035]], device='cuda:0', grad_fn=<SliceBackward0>)
tensor([[-0.2602, -5.1370, -5.6950, -6.2901, -4.2309, -5.6216, -5.7333, -5.7006,
         -5.5627, -4.9309]], device='cuda:0', grad_fn=<SliceBackward0>)
tensor([[-0.2914, -5.2362, -5.8341, -6.4501, -4.3205, -5.6565, -5.8317, -5.7976,
         -5.6003, -5.0518]], device='cuda:0', grad_fn=<SliceBackward0>)
tensor([[-0.4161, -5.7001, -6.2253, -7.0079, -4.7159, -5.7689, -5.9444, -6.3445,
         -5.8075, -5.2991]], device='cuda:0', grad_fn=<SliceBackward0>)
tensor([[-0.2539, -4.9628, -5.5097, -6.2616, -3.9309, -5.6304, -5.6860, -5.7787,
         -5.4036, -4.7867]], device='cuda:0', grad_fn=<SliceBackward0>)
tensor([[-0.3078, -5.0633, -5.5751, -6.2894, -4.1491, -5.4830, -5.6293, -5.7709,
         -5.4242, -4.9041]], device='cuda:0', grad_fn=<SliceBackward0>)
tensor([[-0.2285, -4.7042, -5.5011, -6.3054, -3.3567, -5.3786, -5.7463, -5.6819,
 

...

tensor([[-0.3256, -4.7467, -5.5226, -6.0315, -3.6120, -4.8784, -5.7467, -5.3985,
         -4.4853, -4.4497]], device='cuda:0', grad_fn=<SliceBackward0>)
tensor([[-0.1604, -4.6495, -5.0257, -4.8603, -3.7531, -4.4853, -4.5935, -4.7431,
         -4.7898, -4.1312]], device='cuda:0', grad_fn=<SliceBackward0>)
tensor([[-0.2011, -4.7939, -5.8845, -6.4036, -3.5221, -5.6201, -5.9370, -5.6149,
         -5.3307, -4.7270]], device='cuda:0', grad_fn=<SliceBackward0>)
tensor([[-0.2282, -5.0049, -5.5952, -5.9616, -3.8251, -4.3600, -5.2732, -5.5117,
         -4.2437, -4.1388]], device='cuda:0', grad_fn=<SliceBackward0>)
tensor([[-0.1410, -4.7290, -5.2521, -5.3431, -3.6721, -4.2591, -4.9208, -4.9867,
         -4.4040, -4.0813]], device='cuda:0', grad_fn=<SliceBackward0>)
tensor([[-0.2201, -4.9078, -5.5429, -5.9850, -3.6293, -4.4272, -5.2733, -5.4428,
         -4.1365, -4.2228]], device='cuda:0', grad_fn=<SliceBackward0>)
tensor([[-0.2055, -4.8129, -5.6586, -6.0338, -3.7320, -4.8030, -5.5494, -5.3487,
 

tensor([[-0.1490, -4.7679, -5.2585, -5.6320, -3.6457, -4.3069, -5.1345, -5.1765,
         -4.2887, -4.1578]], device='cuda:0', grad_fn=<SliceBackward0>)
tensor([[-0.2349, -4.9337, -5.8907, -6.0542, -3.6422, -4.5634, -5.3968, -5.4216,
         -4.3674, -4.4106]], device='cuda:0', grad_fn=<SliceBackward0>)
tensor([[-0.2898, -5.0224, -5.7501, -6.2950, -3.7263, -4.7056, -5.5962, -5.5940,
         -4.3365, -4.3014]], device='cuda:0', grad_fn=<SliceBackward0>)
tensor([[-0.2423, -4.8517, -5.7200, -6.0636, -3.7068, -4.4787, -5.3628, -5.4818,
         -4.1695, -4.2751]], device='cuda:0', grad_fn=<SliceBackward0>)
tensor([[-0.2238, -4.9241, -5.4899, -5.9317, -3.6428, -4.4703, -5.3421, -5.4226,
         -4.1710, -4.1637]], device='cuda:0', grad_fn=<SliceBackward0>)
tensor([[-0.1897, -4.9530, -5.5627, -6.3727, -3.7552, -4.7011, -5.5736, -5.6917,
         -4.3356, -4.4248]], device='cuda:0', grad_fn=<SliceBackward0>)
tensor([[-0.1467, -4.8851, -5.4005, -5.7626, -3.6597, -4.3182, -5.1509, -5.2719,
 

tensor([[-0.2336, -4.8554, -5.6336, -6.1990, -3.5171, -4.7749, -5.5371, -5.5675,
         -4.2413, -4.3546]], device='cuda:0', grad_fn=<SliceBackward0>)
tensor([[-0.1792, -4.8673, -5.4203, -5.8823, -3.6485, -4.4398, -5.2470, -5.3493,
         -4.1706, -4.1224]], device='cuda:0', grad_fn=<SliceBackward0>)
tensor([[-0.2433, -4.9539, -5.5447, -5.9624, -3.7159, -4.4562, -5.3925, -5.4866,
         -4.2073, -4.0869]], device='cuda:0', grad_fn=<SliceBackward0>)
tensor([[-0.1166, -4.5603, -5.0814, -5.0383, -3.5855, -4.2988, -4.6446, -4.6460,
         -4.5819, -4.0037]], device='cuda:0', grad_fn=<SliceBackward0>)
tensor([[-0.1717, -4.9921, -5.4401, -5.9393, -3.6620, -4.4516, -5.3173, -5.4228,
         -4.2537, -4.1674]], device='cuda:0', grad_fn=<SliceBackward0>)
tensor([[-0.2482, -4.8953, -5.5321, -5.9465, -3.6648, -4.5179, -5.4039, -5.4163,
         -4.1487, -4.1565]], device='cuda:0', grad_fn=<SliceBackward0>)
tensor([[-0.1683, -4.8681, -5.4148, -5.7307, -3.6769, -4.3274, -5.1869, -5.2800,
 

tensor([[-0.2340, -4.8259, -5.6800, -6.4016, -3.4322, -4.9644, -5.6516, -5.7037,
         -4.4982, -4.3799]], device='cuda:0', grad_fn=<SliceBackward0>)
tensor([[-0.2028, -4.8247, -5.7923, -5.7864, -3.6883, -4.4629, -5.1065, -5.1959,
         -4.3235, -4.3601]], device='cuda:0', grad_fn=<SliceBackward0>)
tensor([[-0.2454, -4.9524, -5.5948, -5.9341, -3.7240, -4.4052, -5.3956, -5.4705,
         -4.2220, -4.2630]], device='cuda:0', grad_fn=<SliceBackward0>)
tensor([[-0.2374, -4.8608, -5.6340, -6.0075, -3.5944, -4.6201, -5.5190, -5.4255,
         -4.2763, -4.3002]], device='cuda:0', grad_fn=<SliceBackward0>)
tensor([[-0.1965, -4.8941, -6.0909, -6.5713, -3.5969, -5.3376, -5.7984, -5.6479,
         -4.9773, -4.7902]], device='cuda:0', grad_fn=<SliceBackward0>)
tensor([[-0.4036, -5.0878, -5.4515, -6.2815, -4.0730, -5.8196, -5.9324, -6.3812,
         -5.6422, -4.7925]], device='cuda:0', grad_fn=<SliceBackward0>)
tensor([[-0.2522, -4.9048, -5.6188, -6.0252, -3.6818, -4.4233, -5.3453, -5.5020,
 

tensor([[-0.2624, -4.8591, -5.7339, -6.4146, -3.4212, -4.9832, -5.6642, -5.6529,
         -4.4155, -4.3317]], device='cuda:0', grad_fn=<SliceBackward0>)
tensor([[-0.2417, -4.7679, -5.5837, -6.4131, -3.4073, -5.2875, -5.7188, -5.8311,
         -4.8499, -4.4665]], device='cuda:0', grad_fn=<SliceBackward0>)
tensor([[-0.1648, -4.9381, -5.4586, -5.8862, -3.6650, -4.3887, -5.3066, -5.3883,
         -4.2583, -4.1198]], device='cuda:0', grad_fn=<SliceBackward0>)
tensor([[-0.2398, -4.7566, -5.5781, -5.9216, -3.4288, -4.9681, -5.6946, -5.4815,
         -4.4530, -4.4661]], device='cuda:0', grad_fn=<SliceBackward0>)
tensor([[-0.1574, -4.9656, -5.4556, -6.0090, -3.6873, -4.4593, -5.3443, -5.5004,
         -4.2363, -4.1681]], device='cuda:0', grad_fn=<SliceBackward0>)
tensor([[-0.1171, -4.7311, -5.2570, -5.4946, -3.6858, -4.2460, -4.9522, -5.0578,
         -4.3151, -4.0795]], device='cuda:0', grad_fn=<SliceBackward0>)
tensor([[-0.1474, -4.6306, -5.1263, -4.9367, -3.6675, -4.3168, -4.5841, -4.6134,
 

In [92]:
label1=np.hstack(label1)
label2=np.hstack(label2)

mu_real_All=np.vstack(mu_real_All)
embedding=dis_evaluator.reduce_latent2d(dis_evaluator.normalize_matrix(mu_real_All), dim_red='umap')

LatentZ=pd.DataFrame(mu_real_All)
#LatentZ.to_csv('C:/Users/Kevin Tsia/VIA/Mix_Latent_LCtry.csv')
LabelAll=pd.DataFrame(label)
#LabelAll.to_csv('C:/Users/Kevin Tsia/VIA/Mix_Label_Liv.csv')



In [93]:
sns.set(font_scale = 5)

#data_real=np.hstack((embedding, labelAll[:]))
df1 = pd.DataFrame(embedding, columns=['col1','col2'])
df2 = pd.DataFrame(label1, columns=['ClassLabels1'])
df3 = pd.DataFrame(label2, columns=['ClassLabels2'])
df_F=pd.concat([df1, df2, df3],axis=1)

In [94]:
df2

,ClassLabels1
0,H1975
1,H1975
2,H1975
3,H1975
4,H1975
...,...
5992,H526
5993,H526
5994,H526
5995,H526


In [96]:
plt.figure(figsize=(10,10))
#plt.axis('off')
sns.scatterplot(data=df_F, x="col1", y="col2", s=50, hue="ClassLabels1", palette=('Set1'))
plt.legend(markerscale=5, fontsize=30)